In [1]:
import os, time, pickle, sys, math
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
from pymol import cmd
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
import pymesh

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from scipy.cluster.hierarchy import fcluster, linkage, single
from scipy.spatial.distance import pdist

In [2]:
import pymol

### Step 0: load PDBbind data

In [3]:
pdbbind_table = pd.read_csv('index/INDEX_general_PL_name.2020', sep='  ', skiprows=6, header=None)
pdbbind_table.columns = ['pdbid', 'year', 'uniprot', 'name']
pdbbind_table.head(3)

/data/lishuya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


pdbid  year uniprot                                          name
0  6mu1  2018  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1
1  3t8s  2011  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1
2  1n4k  2002  P11881  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1

### get protein sequence

In [4]:
def get_seq(pdbid, chains):
    cmd.reinitialize()
    cmd.load("../outputPDB/01-benchmark_pdbs/{}_{}.pdb".format(pdbid, chains))
    cmd.load("../PDBbind2020/pdbbind/{}/{}_ligand.sdf".format(pdbid, pdbid))
    cmd.select('seq', '{}_ligand expand 8 and {}_{}'.format(pdbid, pdbid, chains))
    dict_info = defaultdict(lambda :0)
    cmd.iterate('seq', 'dict_info[chain] += 1', space=locals())
    dict_info = dict(dict_info)
    
    fastastr = cmd.get_fastastr('all')
    dict_seq = {}
    name = ""
    seq = ""
    for line in fastastr.split('\n'):
        if len(line) == 0:
            continue
        if line[0] == '>':
            if name != "":
                dict_seq[name] = seq
            name = line[1:].strip()
            seq = ""
        else:
            seq += line
    dict_seq[name] = seq
    
    list_remove = []
    for chain in dict_info:
        name = '{}_{}_{}'.format(pdbid, chains, chain)
        seq = dict_seq[name]
        
        if (seq.count("?") / len(seq)) > 0.50:
            dict_seq.pop(name)
            list_remove.append(chain)
    for chain in list_remove:
        dict_info.pop(chain)
        
    chain = ""
    max_value = 0
    for key, value in dict_info.items():
        if value > max_value:
            chain = key
            max_value = value
            
    name = '{}_{}_{}'.format(pdbid, chains, chain)
    if name in dict_seq:
        return dict_seq[name]
    else:
        return None

In [5]:
with open('../list_task', 'rb') as f:
    list_task = pickle.load(f)
list_task[:10]

['3ehx_A -2.292_-4.242_-5.148',
 '1zz2_A 33.006_74.208_18.441',
 '5m6m_A -14.204_-18.755_-5.404',
 '3ds6_A 31.704_26.103_62.549',
 '2wks_ABCDEF 3.953_14.697_-22.198',
 '4mho_A 35.105_24.185_31.026',
 '3t6y_ABCF -7.532_26.641_-3.662',
 '5ito_A 19.472_-7.039_-5.581',
 '5fnq_A 17.216_64.595_27.844',
 '1gi4_A 42.854_-2.139_29.450']

In [6]:
dict_seq = {}
for i, item in enumerate(list_task):
    print(i, '\r', end="")
    pdbid_chains, x = item.split(' ')
    pdbid, chains = pdbid_chains.split('_')
    try:
        seq = get_seq(pdbid, chains).replace('?', 'X')
        if seq is None:
            continue
        dict_seq[pdbid] = seq
    except:
        pass

 PyMOL not running, entering library mode (experimental)


In [7]:
len(dict_seq)

19393

In [8]:
pickle.dump(dict_seq, open('dict_seq', 'wb'))

In [9]:
dict_seq = pickle.load(open('dict_seq', 'rb'))

In [10]:
seq_len = np.array([len(seq) for pdbid,seq in dict_seq.items()])
seq_X = np.array([seq.count("X") for pdbid,seq in dict_seq.items()])
unique_seq, unique_idx = np.unique(list(dict_seq.values()), return_inverse=True)
print(len(unique_seq), len(unique_idx))
seq_2_id = {seq:i for i, seq in enumerate(unique_seq)}

12097 19393


In [14]:
print('min len', seq_len.min())
print('most X', (seq_X / seq_len).max())

min len 24
most X 0.1390728476821192


In [12]:
np.array(list(dict_seq.keys()))[(seq_X / seq_len) > 0.1]

array(['6d6t'], dtype='<U4')

In [13]:
unique_seq[np.array([(seq.count('X') / len(seq)) > 0.1 for seq in unique_seq])]

array(['GDVTVILNNLLEGYDNKLRPDIGVKPTLIHTDMYVNSIGPVNAINMEYTIDIFFAQTWYDRRLKFNSTIKVLRLNSNMVGKIWIPDTFFRNSKKADAHWITTPNRMLRIWNDGRVLYTLRLTIDAECQLQLHNDEHSCPLEFSSYGYPREEIVYQWKRSSVEVGDTRSWRLYQFSFVGLRNTTEVVKTTSGDYVVMSVYFDLSRRMGYFTIQTYIPCTLIVVLSWVSFWINKDAVPARTSLGITTVLTMTTLSTIARKSLXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'],
      dtype='<U1718')

In [76]:
# np.array(list(dict_seq.keys()))[np.array(list(dict_seq.values())) == 'GDQNATXG']

In [17]:
12097  / 350

34.56285714285714

In [18]:
fw = open('./intermediate_outputs/represent_pocket_seqs.fasta', 'w')
for i, seq in enumerate(unique_seq):
    fw.write('>'+str(i)+'\n'+seq+'\n')
fw.close()

In [19]:
list_f = []
for l in range(35):
    list_f.append(open('./intermediate_outputs/represent_pocket_seqs_{}.fasta'.format(l), 'w'))
for i, seq in enumerate(unique_seq):
    l = i // 350
    list_f[l].write('>'+str(i)+'\n'+seq+'\n')
for f in list_f:
    f.close()

In [20]:
f = open('sh05_align.sh', 'w')
f.write('L=$1\ntouch ./tmp_sh/sh05_running_$L\n\
export LD_LIBRARY_PATH=$PWD/smith-waterman-src/:$LD_LIBRARY_PATH\n\
./smith-waterman-src/ssw_test -p ./intermediate_outputs/represent_pocket_seqs_$L.fasta ./intermediate_outputs/represent_pocket_seqs.fasta > ./intermediate_outputs/align_$L.out\n\
rm  ./tmp_sh/sh05_running_$L\n')
f.close()

In [21]:
for l in range(35):
    os.system('sh ./sh05_align.sh {} &'.format(l))

# Wait

In [22]:
os.system('cat ./intermediate_outputs/align_* > ./intermediate_outputs/align.out')

0

In [23]:
def check_symmetric(a, tol=1e-3):
    return np.allclose(a, a.T, atol=tol)

def get_protein_vocabulary_dict(fasta_name):
    protein_vocabulary_dict = {}
    f = open(fasta_name)
    i = 0
    for line in f.readlines():
        if line[0] == '>':
            protein_vocabulary_dict[line[1:-1]] = i
            i += 1
    f.close()
    print('protein_vocabulary_dict', len(protein_vocabulary_dict))
    return protein_vocabulary_dict

def sim_aquire(target_name, query_name, output_name):
    '''without normalization'''
    target_dict = get_protein_vocabulary_dict(target_name)
    query_dict = get_protein_vocabulary_dict(query_name)
    p_simi = np.zeros((len(target_dict), len(query_dict)))
    
    # parse results
    count = 0
    with open(output_name, 'r') as f:
        line = f.readline()
        while line:
            count += 1
            if count %100000 == 0:
                print('processed lines', count // 10000, '\r', end='')
            a = line.strip('\n').split(' ')[-1]
            b = f.readline().strip('\n').split(' ')[-1]
            c = float(int(f.readline().strip('\n').split( )[1]))
            d = f.readline()
            p_simi[target_dict[a], query_dict[b]] = c
            line = f.readline()
    
    assert check_symmetric(p_simi)
    
    
#     f = open(output_name)
#     lines = f.readlines()
#     f.close()
#     print('total lines', len(lines))
#     for i in range(0,len(lines),4):
#         try:
#             a = lines[i].strip('\n').split(' ')[-1]
#             b = lines[i+1].strip('\n').split(' ')[-1]
#             c = float(int(lines[i+2].strip('\n').split( )[1]))
#             p_simi[target_dict[a], query_dict[b]] = c
#         except:
#             print('wrong', i, a, b, c)
#             print('wrong', i, a, b, c, target_dict[a], query_dict[b])
#             xxx
#     assert check_symmetric(p_simi)
    
    # normalize
    for i in range(p_simi.shape[0]):
        for j in range(p_simi.shape[0]):
            if i == j:
                continue
            p_simi[i,j] = p_simi[i,j] / (float(np.sqrt(p_simi[i,i])*np.sqrt(p_simi[j,j]))+1e-12)
    for i in range(len(p_simi)):
        p_simi[i,i] = p_simi[i,i] / float(np.sqrt(p_simi[i,i])*np.sqrt(p_simi[i,i]))
    print('p_simi', p_simi.shape)
    assert check_symmetric(p_simi)
    
    target_list = ['']*len(target_dict)
    for pid, idx in target_dict.items():
        target_list[idx] = pid
    print('target_list', len(target_list))
    
    query_list = ['']*len(query_dict)
    for pid, idx in query_dict.items():
        query_list[idx] = pid
    print('query_list', len(query_list))
    assert target_list == query_list
    return p_simi, target_list

prot_sim_mat, pid_list = sim_aquire('./intermediate_outputs/represent_pocket_seqs.fasta', 
                                    './intermediate_outputs/represent_pocket_seqs.fasta',
                                    './intermediate_outputs/align.out')
prot_sim_mat[np.where(prot_sim_mat>1)] = 1

protein_vocabulary_dict 12097
protein_vocabulary_dict 12097
p_simi (12097, 12097) 0 1140   3350      7710  7970    9910   13570  
target_list 12097
query_list 12097


In [24]:
def protein_clustering(pid_list, sim_mat):
    cluster_dict = {}
    print('start protein clustering...')
    P_dist = []
    for i in range(sim_mat.shape[0]):
        P_dist += (1-sim_mat[i,(i+1):]).tolist()
    P_dist = np.array(P_dist)
    P_link = single(P_dist)
    for thre in [0.3, 0.4, 0.5, 0.6]:
        P_clusters = fcluster(P_link, thre, 'distance')
        len_list = []
        for i in range(1,max(P_clusters)+1):
            len_list.append(P_clusters.tolist().count(i))
        print('thre', thre, 'total num of proteins', len(pid_list), 'num of clusters', max(P_clusters), 'max length', max(len_list))
        P_cluster_dict = {pid_list[i]:P_clusters[i] for i in range(len(pid_list))}
        cluster_dict[thre] = P_cluster_dict
    return cluster_dict

p_cluster_dict = protein_clustering(pid_list, prot_sim_mat)

start protein clustering...
thre 0.3 total num of proteins 12097 num of clusters 3225 max length 202
thre 0.4 total num of proteins 12097 num of clusters 2977 max length 206
thre 0.5 total num of proteins 12097 num of clusters 2698 max length 256
thre 0.6 total num of proteins 12097 num of clusters 2274 max length 786


In [25]:
for thre in [0.3, 0.4, 0.5, 0.6]:
    count_error = 0
    cluster_list = []
    for i in pdbbind_table.index:
        pdbid = pdbbind_table.loc[i, 'pdbid']
        try:
            seq = dict_seq[pdbid]
            pid = str(seq_2_id[seq])
            cluster_list.append(p_cluster_dict[thre][pid])
        except:
            print('no sequence error:', pdbid)
            count_error += 1
            cluster_list.append(max(p_cluster_dict[thre].values())+1)
    print('cluster_list', thre, len(cluster_list))
    print('count_error', count_error)
    pdbbind_table['p_group_'+str(thre)] = cluster_list
pdbbind_table[:3]

no sequence error: 6mu1
no sequence error: 4qsh
no sequence error: 1bxr
no sequence error: 3t09
no sequence error: 5yij
no sequence error: 2z5o
no sequence error: 6mck
no sequence error: 2f2h
no sequence error: 2x6k
no sequence error: 3buw
no sequence error: 2x6x
no sequence error: 6gg4
no sequence error: 4o2a
no sequence error: 1p0y
no sequence error: 4u91
no sequence error: 5bjt
no sequence error: 3m17
no sequence error: 5n70
no sequence error: 3bux
no sequence error: 3bun
no sequence error: 3bum
no sequence error: 4j8g
no sequence error: 4z90
no sequence error: 6hv7
no sequence error: 6hvw
no sequence error: 4qls
no sequence error: 5v13
no sequence error: 4uu7
no sequence error: 5mlw
no sequence error: 5mlo
no sequence error: 5trr
no sequence error: 5ts0
no sequence error: 5try
no sequence error: 5trg
no sequence error: 6ocz
no sequence error: 4bv2
no sequence error: 2hug
no sequence error: 4u2w
no sequence error: 5fsb
no sequence error: 5fsc
no sequence error: 2r1w
no sequence erro

pdbid  year uniprot                                          name  \
0  6mu1  2018  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   
1  3t8s  2011  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   
2  1n4k  2002  P11881  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   

   p_group_0.3  p_group_0.4  p_group_0.5  p_group_0.6  
0         3226         2978         2699         2275  
1          152          141          133          114  
2          151          141          133          114

In [26]:
pdbbind_table.shape

(19443, 8)

In [27]:
pdbbind_table.to_csv('./intermediate_outputs/pdbbind_table_1.1.tsv', sep='\t', index=None)

### get mol

In [28]:
def get_mol(pdbid):

    sdf_path = "../PDBbind2020/pdbbind/{}/{}_ligand.sdf".format(pdbid, pdbid)
    mol2_path = "../PDBbind2020/pdbbind/{}/{}_ligand.mol2".format(pdbid, pdbid)
    
    if os.path.exists(sdf_path):
        mol = Chem.MolFromMol2File(mol2_path)
        if mol is None:
            mol = Chem.SDMolSupplier(sdf_path)[0]
        if mol is None:
            mol = Chem.MolFromMol2File(mol2_path, sanitize=False)
        if mol is None:
            mol = Chem.SDMolSupplier(sdf_path, sanitize=False)[0]
    else:
        mol = None
    return mol

In [29]:
success_list = []
failed_list = []
mol_dict = {}
for i in pdbbind_table.index:
    pdbid = pdbbind_table.loc[i, 'pdbid']
    mol = get_mol(pdbid)
#     if mol is None:
#         break
    
    if mol is not None:
        success_list.append(i)
        mol_dict[pdbid] = mol
    else:
        failed_list.append(i)

RDKit WARNING: [23:38:55] 6mu1_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 3t8s_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 1n4k_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 4wz6_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 1t3t_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 4qsh_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 4qsk_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 1bxr_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 5a3r_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 5a3q_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 5a3s_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:38:55] Explicit valence for atom # 25

RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'Du' not found
RDKit ERROR: Violation occurred on line 91 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:38:55] ERROR: Element 'Du' not found
RDKit WARNING: [23:38:55] Warning - O.co2 with degree >1.
RDKit ERROR: [23:38:55] ERROR: moving to the beginning of the next molecule
RDKit ERROR: [23:38:55] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'Du' not found
RDKit ERROR: Violation occurred on line 91 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:38:55] ERROR: Element 'Du' not found
RDKit WARNING: [23:38:55] 4p4d_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:55] 4p4j_li

RDKit WARNING: [23:38:56] 4pl0_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:38:56] ERROR: Can't kekulize mol.  Unkekulized atoms: 2
RDKit ERROR: 
RDKit ERROR: [23:38:56] Can't kekulize mol.  Unkekulized atoms: 3 4 5 15 17
RDKit ERROR: 
RDKit WARNING: [23:38:56] sanitise [23:38:56] 5tyk_ligand: [23:38:56] 2hwg_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
RDKit WARNING: [23:38:56] 3au6_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:38:56] Can't kekulize mol.  Unkekulized atoms: 20 21 22 25 29
RDKit ERROR: 
RDKit ERROR: [23:38:56] ERROR: Could not sanitize molecule ending on line 97
RDKit WARNING: [23:38:56] 3au6_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:56] 5ods_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:56] 6egs_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:56] 3iaf_ligand: warning - O.co2

RDKit WARNING: [23:38:58] 3cph_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:38:58] ERROR: Can't kekulize mol.  Unkekulized atoms: 18 19 20 23 27
RDKit ERROR: 
RDKit ERROR: [23:38:58] Can't kekulize mol.  Unkekulized atoms: 18 19 20 23 27
RDKit ERROR: 
RDKit ERROR: [23:38:58] ERROR: Could not sanitize molecule ending on line 91
RDKit WARNING: [23:38:58] 3cph_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:58] 6npf_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:58] 6d3q_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:58] 1i43_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:58] 1i41_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:58] 5v4b_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:58] 2ovq_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:58] 3upy_ligand: war

RDKit WARNING: [23:38:59] 1y8o_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:59] 1y8p_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:59] 3v4t_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:59] 3upk_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:59] 4aq4_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:59] 1ent_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:38:59] Can't kekulize mol.  Unkekulized atoms: 23 24 25 26 27
RDKit ERROR: 
RDKit ERROR: [23:38:59] ERROR: Could not sanitize molecule ending on line 198
RDKit WARNING: [23:38:59] 1ent_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:38:59] 1gvw_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:38:59] ERROR: Can't kekulize mol.  Unkekulized atoms: 23 24 25 26 27
RDKit ERROR: 
RDKit ERROR: [23:38:59] Can't kekulize mo

...

RDKit WARNING: [23:39:12] 5y61_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:39:12] ERROR: Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 12
RDKit ERROR: 
RDKit ERROR: [23:39:12] Can't kekulize mol.  Unkekulized atoms: 14 15 16 19 23
RDKit ERROR: 
RDKit ERROR: [23:39:12] ERROR: Could not sanitize molecule ending on line 81
RDKit WARNING: [23:39:12] 5y61_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:12] 5y62_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:39:12] ERROR: Can't kekulize mol.  Unkekulized atoms: 14 15 16 19 23
RDKit ERROR: 
RDKit ERROR: [23:39:12] Can't kekulize mol.  Unkekulized atoms: 14 15 16 19 23
RDKit ERROR: 
RDKit ERROR: [23:39:12] ERROR: Could not sanitize molecule ending on line 81
RDKit WARNING: [23:39:12] 5y62_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:12] 1j4k_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:12

RDKit WARNING: [23:39:13] 5yz2_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:13] 1h6h_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:13] 4ozl_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:13] 4ozj_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:13] 4ozn_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:13] 3clp_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:13] 4muv_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:39:13] ERROR: Can't kekulize mol.  Unkekulized atoms: 15
RDKit ERROR: 
RDKit ERROR: [23:39:13] Can't kekulize mol.  Unkekulized atoms: 13 14 15 18 22
RDKit ERROR: 
RDKit ERROR: [23:39:13] ERROR: Could not sanitize molecule ending on line 76
RDKit WARNING: [23:39:13] 4muv_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:13] 2k0g_ligand: warning - O.co2

RDKit WARNING: [23:39:14] 2w5i_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 1w4p_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 3lxo_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 1jn4_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 1o0o_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 1jvu_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 1o0m_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 6pvu_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
RDKit WARNING: [23:39:14] 6pvu_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 1w4o_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:14] 1w4q_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: 

RDKit ERROR: [23:39:15] ERROR: Can't kekulize mol.  Unkekulized atoms: 87 88 89 90 91
RDKit ERROR: 
RDKit ERROR: [23:39:15] Explicit valence for atom # 6 C, 5, is greater than permitted
RDKit ERROR: [23:39:15] Explicit valence for atom # 6 C, 5, is greater than permitted
RDKit ERROR: [23:39:15] ERROR: Could not sanitize molecule ending on line 300
RDKit ERROR: [23:39:15] ERROR: Explicit valence for atom # 6 C, 5, is greater than permitted
RDKit ERROR: [23:39:15] Explicit valence for atom # 9 C, 5, is greater than permitted
RDKit ERROR: [23:39:15] Explicit valence for atom # 9 C, 5, is greater than permitted
RDKit ERROR: [23:39:15] ERROR: Could not sanitize molecule ending on line 295
RDKit WARNING: [23:39:15] sanitise [23:39:15] 1qpf_ligand: [23:39:15] sanitise [23:39:15] 1a7x_ligand: [23:39:15] 1btn_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:15] 5tq1_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:15] 5tqs_ligand: warning 

RDKit WARNING: [23:39:16] 3qo3_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:16] 2ylc_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:16] 5f29_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:16] 2k7l_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit ERROR: [23:39:15] ERROR: Can't kekulize mol.  Unkekulized atoms: 14 15 16 19 23
RDKit ERROR: 
RDKit ERROR: [23:39:16] Can't kekulize mol.  Unkekulized atoms: 66 67 68 69 70
RDKit ERROR: 
RDKit ERROR: [23:39:16] ERROR: Could not sanitize molecule ending on line 658
RDKit WARNING: [23:39:16] 2k7l_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:16] 2p0x_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [23:39:16] 1w70_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
RDKit WARNING: [23:39:16] 2n8t_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RD

In [30]:
print("success_list", len(success_list), 'failed_list', len(failed_list), 'mol_dict', len(mol_dict))

success_list 19439 failed_list 4 mol_dict 19439


In [143]:
with open('casf_mol_dict', 'wb') as f:
    pickle.dump(mol_dict,f)

In [144]:
print('mol_dict', len(mol_dict))

mol_dict 19439


In [33]:
print("success_list", len(success_list), 'failed_list', len(failed_list), 'mol_dict', len(mol_dict))
pdbbind_table = pdbbind_table.iloc[success_list]
print(pdbbind_table.shape)
pdbbind_table[:3]

success_list 19439 failed_list 4 mol_dict 19439
(19439, 8)


pdbid  year uniprot                                          name  \
0  6mu1  2018  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   
1  3t8s  2011  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   
2  1n4k  2002  P11881  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   

   p_group_0.3  p_group_0.4  p_group_0.5  p_group_0.6  
0         3226         2978         2699         2275  
1          152          141          133          114  
2          151          141          133          114

In [34]:
def get_fps(mol_list):
    fps = []
    idx = []
    for i, mol in enumerate(mol_list):
        print('fingerprint:', i, '\r', end='')
        try:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useChirality=True)
            fps.append(fp)
            idx.append(True)
        except:
            idx.append(False)
    print('fingerprint list',len(fps))
    return fps, idx


def calculate_sims(fps1,fps2,simtype='tanimoto'):
    sim_mat = np.zeros((len(fps1),len(fps2)),dtype=np.float32)
    for i in range(len(fps1)):
        if i%10000 == 0:
            print('similarity:', i)
        fp_i = fps1[i]
        if simtype == 'tanimoto':
            sims = DataStructs.BulkTanimotoSimilarity(fp_i,fps2)
        elif simtype == 'dice':
            sims = DataStructs.BulkDiceSimilarity(fp_i,fps2)
        sim_mat[i,:] = sims
    print('compound sim mat', sim_mat.shape)
    return sim_mat


def compound_clustering(fps):
    cluster_dict = {}
    print ('start compound clustering...')
    C_dist = pdist(fps, 'jaccard')
    C_link = single(C_dist)
    for thre in [0.3, 0.4, 0.5, 0.6]:
        C_clusters = fcluster(C_link, thre, 'distance')
        len_list = []
        for i in range(1,max(C_clusters)+1):
            len_list.append(C_clusters.tolist().count(i))
        print('thre', thre, 'total num of compounds', len(fps), 'num of clusters', max(C_clusters), 'max length', max(len_list))
        #C_cluster_dict = {cid_list[i]:C_clusters[i] for i in range(len(cid_list))}
        cluster_dict[thre] = C_clusters
    return cluster_dict

In [35]:
mol_list = [mol_dict[pdbid] for pdbid in pdbbind_table['pdbid']]
print('mol_list', len(mol_list))

fps, idx = get_fps(mol_list)
cluster_dict = compound_clustering(fps)

mol_list 19439
fingerprint: 913 

RDKit ERROR: [23:39:16] ERROR: Can't kekulize mol.  Unkekulized atoms: 66 67 68 69 70
RDKit ERROR: 
RDKit ERROR: [23:39:19] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:19] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:19] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitH

RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:19] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:19] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [

fingerprint: 2244

fingerprint:

...

RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:26] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:26] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [

fingerprint:

fingerprint:

RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:26] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:26] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called without preceding call to calcImplicitValence()
RDKit ERROR: Violation occurred on line 187 in file /opt/conda/conda-bld/rdkit_1591929907781/work/Code/GraphMol/Atom.cpp
RDKit ERROR: Failed Expression: d_implicitValence > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [23:39:26] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Pre-condition Violation
RDKit ERROR: getNumImplicitHs() called wi

fingerprint list 19137
start compound clustering...
thre 0.3 total num of compounds 19137 num of clusters 11159 max length 1085
thre 0.4 total num of compounds 19137 num of clusters 8562 max length 1886
thre 0.5 total num of compounds 19137 num of clusters 5834 max length 5022
thre 0.6 total num of compounds 19137 num of clusters 3141 max length 12526


In [36]:
pdbbind_table = pdbbind_table[np.array(idx)]

In [37]:
print(pdbbind_table.shape)

(19137, 8)


In [38]:
for thre in [0.3, 0.4, 0.5, 0.6]:
    cluster_list = cluster_dict[thre]
    pdbbind_table['c_group_'+str(thre)] = cluster_list
pdbbind_table[:3]

pdbid  year uniprot                                          name  \
0  6mu1  2018  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   
1  3t8s  2011  P29994  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   
2  1n4k  2002  P11881  INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1   

   p_group_0.3  p_group_0.4  p_group_0.5  p_group_0.6  c_group_0.3  \
0         3226         2978         2699         2275         7453   
1          152          141          133          114         6791   
2          151          141          133          114         6791   

   c_group_0.4  c_group_0.5  c_group_0.6  
0         5648         3530         1493  
1         5247         3451         1493  
2         5247         3451         1493

In [39]:
pdbbind_table.to_csv('./intermediate_outputs/pdbbind_table_1.2.tsv', sep='\t', index=None)

### remove multiple ligands

In [54]:
pdbbind_table = pd.read_table('./intermediate_outputs/pdbbind_table_1.2.tsv')
print(pdbbind_table.shape)

(19137, 12)


In [55]:
success_list = []
for i in pdbbind_table.index:
    pdbid = pdbbind_table.loc[i, 'pdbid']
    mol = mol_dict[pdbid]
    smiles = Chem.MolToSmiles(mol)
    if '.' not in smiles:
        success_list.append(i)
        
print("success_list", len(success_list))
pdbbind_table = pdbbind_table.loc[success_list]

success_list 19131


In [56]:
print(pdbbind_table.shape[0])

19131


### remove empty cases

In [57]:
pdbid2chains = {}
for i, item in enumerate(list_task):
    print(i, '\r', end="")
    pdbid_chains, x = item.split(' ')
    pdbid, chains = pdbid_chains.split('_')
    pdbid2chains[pdbid] = item

In [107]:
success_list = []
for i in pdbbind_table.index:
    pdbid = pdbbind_table.loc[i, 'pdbid']
    pdbid_chains, x = pdbid2chains[pdbid].split(' ')
    _, chains = pdbid_chains.split('_')
    
    if not os.path.exists("../outputAnchor/{}_center_{}_aug_1/am_list.npy".format(pdbid_chains, x)):
        continue
    if not os.path.exists("../outputAnchor/{}_center_{}_aug_1/atom_feature.pk".format(pdbid_chains, x)):
        continue
    
    success_list.append(i)
    
print("success_list", len(success_list))
print("total", pdbbind_table.shape[0])

success_list 19019
total 19021


In [108]:
pdbbind_table = pdbbind_table.loc[success_list]
print(pdbbind_table.shape[0])

19019


In [109]:
pdbbind_table.to_csv('./intermediate_outputs/pdbbind_table_1.3.tsv', sep='\t', index=None)

# 🔥2021年10月9日23点46分

### step 1: get train-test table


In [110]:
casf16_pdbids = []
with open('../../CASF-2016/power_scoring/CoreSet.dat') as f:
    for line in f.readlines()[1:]:
        pdbid = line[:4]
        if pdbid not in []:
            casf16_pdbids.append(pdbid)
len(casf16_pdbids)

285

In [111]:
pdbid_to_protein = {}
pdbid_to_group = {}
pdbid_to_year = {}
for i in pdbbind_table.index:
    pdbid, protein, group, year = pdbbind_table.loc[i, ['pdbid', 'uniprot', 'p_group_0.3', 'year']]
    pdbid_to_protein[pdbid] = protein
    pdbid_to_group[pdbid] = group
    pdbid_to_year[pdbid] = year

print(len(pdbid_to_protein))
print(len(pdbid_to_group))
print(len(pdbid_to_year))

19019
19019
19019


In [112]:
np.unique(list(pdbid_to_year.values()))

array([1982, 1984, 1986, 1987, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019])

In [113]:
# check casf_proteins
casf_proteins = set()
list_failed = []
for pdbid in casf16_pdbids:
    if pdbid in pdbid_to_protein:
        casf_proteins.add(pdbid_to_protein[pdbid])
    else:
        list_failed.append(pdbid)
        print(pdbid)

In [114]:
for pdbid in list_failed:
    pdbid_chains, x = pdbid2chains[pdbid].split(' ')
    _, chains = pdbid_chains.split('_')
    print(pdbid2chains[pdbid], os.path.exists("../outputAnchor/{}_center_{}_aug_1/am_list.npy".format(pdbid_chains, x)))

In [115]:
len(pdbid_to_protein)

19019

In [116]:
for pdbid in pdbid_to_protein:
    if pdbid not in pdbid_to_group:
        pid = pdbid_to_protein[pdbid]
        subtable = pdbbind_table[pdbbind_table['uniprot']==pid]
        pdbid_to_group[pdbid] = subtable['p_group_0.3'].values[0]
print(len(pdbid_to_group))

19019


In [117]:
# use protein ids in pdbbind_table for all casf proteins 
casf_proteins - set(np.unique(pdbbind_table['uniprot'].values).tolist())

set()

In [118]:
# number of casf proteins (official: 57 groups)
print('casf_proteins', len(casf_proteins))

casf_proteins 68


In [119]:
# number of casf groups (should be 57)
casf_groups = set()
for pid in casf_proteins:
    subtable = pdbbind_table[pdbbind_table['uniprot']==pid]
    casf_groups.add(subtable['p_group_0.3'].values[0])

print('casf_groups', len(casf_groups))

casf_groups 58


In [120]:
total_pdbids = np.unique(np.concatenate([np.array(casf16_pdbids), pdbbind_table['pdbid'].values]))
len(total_pdbids)

19019

In [121]:
refined_set = set()
with open('index/INDEX_refined_data.2020') as f:
    for line in f.readlines()[6:]:
        refined_set.add(line[:4])
len(refined_set)

5316

In [122]:
kikd_set = set()
ic50_set = set()
with open('index/INDEX_general_PL_data.2020') as f:
    for line in f.readlines()[6:]:
        if line[25:27] == 'Kd' or line[25:27] == 'Ki':
            kikd_set.add(line[:4])
        elif line[25:27] == 'IC':
            ic50_set.add(line[:4])
        else:
            print(line[25:27])
len(kikd_set), len(ic50_set)

(12253, 7190)

In [123]:
type_list = []
for pdbid in total_pdbids:
    if pdbid in casf16_pdbids:
        type_list.append('test')
    elif pdbid_to_group[pdbid] in casf_groups:
        type_list.append('np')
    elif pdbid_to_year[pdbid] == 2019:
        type_list.append('v2020')
    elif pdbid in refined_set:
        type_list.append('refined')
    elif pdbid in kikd_set:
        type_list.append('kikd')
    elif pdbid in ic50_set:
        type_list.append('ic50')
    else:
        print('no label', pdbid)
len(type_list)

19019

In [124]:
group_list = [pdbid_to_group[pdbid] for pdbid in total_pdbids]
len(group_list)

19019

In [164]:
def get_pdbid_to_affinity():
    pdbid_to_aff = {}
    with open('index/INDEX_general_PL.2020') as f:
        count_error = 0
        for line in f.readlines():
            if line[0] != '#':
                lines = line.split('/')[0].strip().split('  ')
                pdbid = lines[0]
                if '<=' in lines[3]:
                    measure = '<='
                elif '>=' in lines[3]:
                    measure = '>='
                elif '=' in lines[3]:
                    measure = '='
                elif '>' in lines[3]:
                    measure = '>'
                elif '<' in lines[3]:
                    measure = '<'
                elif '~' in lines[3]:
                    measure = '~'
                else:
                    print (lines[3], measure)
                afftype = lines[3].split(measure)[0]
                assert afftype in ['IC50', 'Kd', 'Ki']
                value = float(lines[3].split(measure)[1][:-2])
                unit = lines[3].split(measure)[1][-2:]
                if unit == 'nM':
                    pvalue = -np.log10(value)+9
                elif unit == 'uM':
                    pvalue = -np.log10(value)+6
                elif unit == 'mM':
                    pvalue = -np.log10(value)+3
                elif unit == 'pM':
                    pvalue = -np.log10(value)+12
                elif unit == 'fM':
                    pvalue = -np.log10(value)+15
                else:
                    print(unit)
#                 pdbid_to_aff[pdbid] = (afftype, measure, pvalue)
                pdbid_to_aff[pdbid] = pvalue
    print ('count_error not = measurement', count_error)
    return pdbid_to_aff
pdbid_to_aff = get_pdbid_to_affinity()
print('pdbid_to_aff', len(pdbid_to_aff))

with open('/data/lishuya/silexon/doghouse2021/CASF-2016/power_scoring/CoreSet.dat') as f:
    for line in f.readlines()[1:]:
        pdbid_to_aff[line[:4]] = float(line[23:29].strip())
len(pdbid_to_aff)

count_error not = measurement 0
pdbid_to_aff 19443


19443

In [165]:
label_list = [pdbid_to_aff[pdbid] for pdbid in total_pdbids]
len(label_list)

19019

In [166]:
casf_table = pd.DataFrame()
casf_table['pdbid'] = total_pdbids
casf_table['type'] = type_list
casf_table['label'] = label_list
casf_table['p_group_0.3'] = group_list

In [167]:
pd.value_counts(casf_table['type'])

kikd       5080
ic50       4780
np         4631
refined    2952
v2020      1291
test        285
Name: type, dtype: int64

In [168]:
casf_table.to_csv('casf2016_table_new_protein.tsv', sep='\t', index=None)

### step 2: define anchors

In [145]:
list_pdbid = casf_table['pdbid'].values
set_pdbid = set(list_pdbid)
len(list_pdbid)

19019

In [146]:
# batch_process

result_dict = {}
for item in list_task:
    pdbid_chains, center = item.split()
    pdbid, chains = pdbid_chains.split('_')
    if pdbid not in set_pdbid:
        continue
    repeat_list = np.load('../outputAnchor/{}_center_{}_aug_1/anchors.npy'.format(pdbid_chains, center))
    result_dict[pdbid] = repeat_list

print(len(result_dict))

19019


In [147]:
with open('casf2016_anchor_dict_thre4', 'wb') as f:
    pickle.dump(result_dict, f)

In [148]:
# def save_centers(pdbid, refid, coords, centers):
#     cmd.reinitialize()
#     cmd.load('../pdb_file/{}_aligned_to_{}.pdb'.format(pdbid, refid))
    
#     for x,y,z in coords:
#         cmd.pseudoatom('pseudo', pos=[x,y,z], vdw=0.01)
        
#     for x,y,z in centers:
#         cmd.pseudoatom('center', pos=[x,y,z], vdw=1)
#     cmd.show('spheres', 'center')
#     cmd.color('yellow', 'center')
#     cmd.save('./check/check2.2_{}_anchors.pse'.format(pdbid))

### Step 3: get compound mol 划掉

In [149]:
# with open('../pdbbind_food/mol_dict', 'rb') as f:
#     mol_dict = pickle.load(f)
# len(mol_dict)

In [150]:
# def get_mol(pdbid):
#     sdf_path = '../../CASF-2016/coreset/{}/{}_ligand.sdf'.format(pdbid, pdbid)
#     mol2_path = '../../CASF-2016/coreset/{}/{}_ligand.mol2'.format(pdbid, pdbid)
    
#     if os.path.exists(sdf_path):
#         mol = Chem.MolFromMol2File(mol2_path)
#         if mol is None:
#             mol = Chem.SDMolSupplier(sdf_path)[0]
#         if mol is None:
#             mol = Chem.MolFromMol2File(mol2_path, sanitize=False)
#         if mol is None:
#             mol = Chem.SDMolSupplier(sdf_path, sanitize=False)[0]
#     else:
#         mol = None
#     return mol

In [151]:
# for pdbid in casf_table['pdbid']:
#     if pdbid not in mol_dict:
#         mol = get_mol(pdbid)
#         if mol is None:
#             print(pdbid)
#         else:
#             mol_dict[pdbid] = mol
# len(mol_dict)

In [152]:
# with open('casf_mol_dict', 'wb') as f:
#     pickle.dump(mol_dict, f)

## step 4 atom features

In [153]:
feature_dict = {}

for item in list_task:
    pdbid_chains, center = item.split()
    pdbid, chains = pdbid_chains.split('_')
    if pdbid not in set_pdbid:
        continue
        
    atom_dict = pickle.load(open('../outputAnchor/{}_center_{}_aug_1/atom_feature.pk'.format(pdbid_chains, center), 'rb'))
#     print(pdbid, atom_dict[0][0].shape)
    feature_dict[pdbid] = atom_dict[0]
len(feature_dict)

19019

In [154]:
# SAVE AS A WHOLE
with open("casf2016_atom_feature_coord_nei_dict_thre4", "wb") as f:
    pickle.dump(feature_dict, f)